In [25]:
import pandas as pd
from datetime import date

ratings_path = '/mnt/hgfs/share/data/MovieLens/ml-20m/ratings.csv'

rt_reader = pd.read_csv(ratings_path, chunksize=5000)
# The data is too large. We only load 5000 for demonstration
rt_data = rt_reader.get_chunk()

# We select the first 10 users for test
user_datas = []
for user_id in range(1, 11):
    user_datas.append(rt_data[rt_data.userId == user_id])

print user_datas[4]

# Make date as index
for idx, user_data in enumerate(user_datas):
    user_data['timestamp'] = user_data['timestamp'].map(date.fromtimestamp)
    user_data = user_data.set_index('timestamp')
    # sort by time
    user_data = user_data.sort_index()
    user_datas[idx] = user_data
    
# Show example of the first user
print user_datas[4]

     userId  movieId  rating  timestamp
451       5        2     3.0  851527569
452       5       11     5.0  851527751
453       5       17     3.0  851526935
454       5       60     3.0  851617589
455       5       62     5.0  851526935
456       5      104     2.0  851526992
457       5      110     4.0  851527569
458       5      140     2.0  851527012
459       5      141     5.0  851526935
460       5      150     5.0  851527514
461       5      224     2.0  851527650
462       5      235     3.0  851527723
463       5      260     5.0  851527608
464       5      282     4.0  851527569
465       5      316     4.0  851617633
466       5      318     5.0  851527608
467       5      350     4.0  851527688
468       5      364     5.0  851527569
469       5      367     3.0  851527650
470       5      368     5.0  851527650
471       5      370     4.0  851527608
472       5      376     4.0  851617474
473       5      377     5.0  851527723
474       5      380     5.0  851527515


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


            userId  movieId  rating
timestamp                          
1996-12-25       5        2     3.0
1996-12-25       5      500     5.0
1996-12-25       5      508     5.0
1996-12-25       5      529     4.0
1996-12-25       5      531     5.0
1996-12-25       5      587     4.0
1996-12-25       5      588     5.0
1996-12-25       5      589     5.0
1996-12-25       5      590     5.0
1996-12-25       5      593     3.0
1996-12-25       5      595     5.0
1996-12-25       5      608     3.0
1996-12-25       5      491     4.0
1996-12-25       5      631     3.0
1996-12-25       5      671     5.0
1996-12-25       5      708     3.0
1996-12-25       5      720     5.0
1996-12-25       5      736     5.0
1996-12-25       5      780     5.0
1996-12-25       5      788     3.0
1996-12-25       5      832     5.0
1996-12-25       5     1036     5.0
1996-12-25       5     1042     4.0
1996-12-25       5     1073     2.0
1996-12-25       5     1097     5.0
1996-12-25       5      648 

In [3]:
movies_path = '/mnt/hgfs/share/data/MovieLens/ml-20m/movies.csv'

movie_datas = pd.read_csv(movies_path).set_index('movieId')

movie_datas.head()

,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy


In [4]:
# Get the set of all genres
all_genres = set()
for idx, row in movie_datas.iterrows():
    all_genres.update(row.genres.split('|'))
    
print all_genres

set(['Mystery', 'Drama', 'Western', 'Sci-Fi', 'Horror', 'Film-Noir', 'Crime', 'Romance', 'Fantasy', 'Musical', 'Animation', 'War', 'Adventure', 'Action', '(no genres listed)', 'Comedy', 'Documentary', 'Children', 'Thriller', 'IMAX'])


In [5]:
# Expand the user data to include genres

def expand_user_data(user_data):
    for genre in all_genres:
        user_data[genre] = pd.Series()
    user_data = user_data.fillna(0)
    return user_data
    
for (idx, user_data) in enumerate(user_datas):
    user_datas[idx] = expand_user_data(user_data)
    
user_datas[0].head()

,userId,movieId,rating,Mystery,Drama,Western,Sci-Fi,Horror,Film-Noir,Crime,...,Animation,War,Adventure,Action,(no genres listed),Comedy,Documentary,Children,Thriller,IMAX
timestamp,,,,,,,,,,,,,,,,,,,,,
2004-09-10,1,1304,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2004-09-10,1,1997,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2004-09-10,1,6502,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2004-09-10,1,2288,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2004-09-10,1,1214,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
# For each row, copy the rating to the genre of the movieId
for (idx, user_data) in enumerate(user_datas):
    for (rowid, row) in user_data.iterrows():
        print rowid
        print user_data.loc[rowid, :]
        mv_id = int(row['movieId'])
        genres = movie_datas.loc[mv_id].genres.split('|')
        for genre in genres:
            print user_data.loc[rowid]
            user_data.ix[rowid, genres] += float(row['rating'])
        break
    break
            
user_datas[0].head()

2004-09-10
            userId  movieId  rating  Mystery  Drama  Western  Sci-Fi  Horror  \
timestamp                                                                      
2004-09-10       1     1304     3.0      0.0    0.0     42.0     0.0     0.0   
2004-09-10       1     1997     3.5      0.0    0.0     42.0     0.0     0.0   
2004-09-10       1     6502     3.5      0.0    0.0     42.0     0.0     0.0   
2004-09-10       1     2288     4.0      0.0    0.0     42.0     0.0     0.0   
2004-09-10       1     1214     4.0      0.0    0.0     42.0     0.0     0.0   
2004-09-10       1     1215     4.0      0.0    0.0     42.0     0.0     0.0   
2004-09-10       1     6242     3.5      0.0    0.0     42.0     0.0     0.0   
2004-09-10       1     1219     4.0      0.0    0.0     42.0     0.0     0.0   
2004-09-10       1     4306     4.0      0.0    0.0     42.0     0.0     0.0   
2004-09-10       1     1246     3.5      0.0    0.0     42.0     0.0     0.0   
2004-09-10       1     1258  

,userId,movieId,rating,Mystery,Drama,Western,Sci-Fi,Horror,Film-Noir,Crime,...,Animation,War,Adventure,Action,(no genres listed),Comedy,Documentary,Children,Thriller,IMAX
timestamp,,,,,,,,,,,,,,,,,,,,,
2004-09-10,1,1304,3.0,0.0,0.0,48.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,48.0,0.0,0.0,0.0,0.0,0.0,0.0
2004-09-10,1,1997,3.5,0.0,0.0,48.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,48.0,0.0,0.0,0.0,0.0,0.0,0.0
2004-09-10,1,6502,3.5,0.0,0.0,48.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,48.0,0.0,0.0,0.0,0.0,0.0,0.0
2004-09-10,1,2288,4.0,0.0,0.0,48.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,48.0,0.0,0.0,0.0,0.0,0.0,0.0
2004-09-10,1,1214,4.0,0.0,0.0,48.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,48.0,0.0,0.0,0.0,0.0,0.0,0.0
